In [64]:
import json
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.io import output_notebook
import numpy as np
output_notebook()

Loading BokehJS ...

In [17]:
with open('mqtt_log_reencoded.jsonl') as f:
    data = [json.loads(line) for line in f]

# Normalise the times
start_time = data[0]['timestamp']
for d in data:
    d['timestamp'] -= start_time


In [18]:
# Process the data
motor_speeds = [d for d in data if d['topic'] == 'motors/wheels']
encoders = [d for d in data if d['topic'] == 'sensors/encoders/data']


In [20]:
# Lets plot the motor speed outputs  on Y vs time on x.
times = [d['timestamp'] for d in motor_speeds]
left_speed = [d['json_payload'][0] for d in motor_speeds]
right_speed = [d['json_payload'][1] for d in motor_speeds]
motor_p = figure(title='Motor Speeds', x_axis_label='Time', y_axis_label='Speed')
motor_p.line(times, left_speed, legend_label='Left Wheel Speed', line_width=2)
motor_p.line(times, right_speed, legend_label='Right Wheel Speed', line_width=2, color='red')
show(motor_p)

In [21]:
# Now plot the encoder values on Y vs time on x.
encoders = [d for d in data if d['topic'] == 'sensors/encoders/data']
times = [d['timestamp'] for d in encoders]
left_enc = [d['json_payload']['left_delta'] for d in encoders]
right_enc = [d['json_payload']['right_delta'] for d in encoders]
encoder_p = figure(title='Encoder Values', x_axis_label='Time', y_axis_label='Delta')
encoder_p.line(times, left_enc, legend_label='Left Encoder Delta', line_width=2)
encoder_p.line(times, right_enc, legend_label='Right Encoder Delta', line_width=2, color='red')
show(encoder_p)


In [23]:
# Load the after speed change data
with open('after_speed_change.jsonl') as f:
    data2 = [json.loads(line) for line in f]
# Normalise the times
start_time = data2[0]['timestamp']
for d in data2:
    d['timestamp'] -= start_time


In [70]:
def return_encoders_figure(source_data):
    encoders = [d for d in source_data if d['topic'] == 'sensors/encoders/data']
    times = [d['timestamp'] for d in encoders]
    left_enc = [d['json_payload']['left_delta'] for d in encoders]
    right_enc = [d['json_payload']['right_delta'] for d in encoders]
    encoder_p = figure(title='Encoder Values', x_axis_label='Time', y_axis_label='Delta')
    encoder_p.line(times, left_enc, legend_label='Left Encoder Delta', line_width=2)
    encoder_p.line(times, right_enc, legend_label='Right Encoder Delta', line_width=2, color='red')
    return encoder_p

def return_motors_figure(source_data):
    motor_speeds = [d for d in source_data if d['topic'] == 'motors/wheels']
    times = [d['timestamp'] for d in motor_speeds]
    left_speed = [d['json_payload'][0] for d in motor_speeds]
    right_speed = [d['json_payload'][1] for d in motor_speeds]
    motor_p = figure(title='Motor Speeds', x_axis_label='Time', y_axis_label='Speed', y_range=(-1.2, 1.2))
    
    motor_p.line(times, left_speed, legend_label='Left Wheel Speed', line_width=2)
    motor_p.line(times, right_speed, legend_label='Right Wheel Speed', line_width=2, color='red')
    return motor_p

def truncate_data_after_drive_stop(decoded_lines):
    for line in decoded_lines:
        yield line
        if line['topic'] == 'drive/stop':
            break

def truncate_data_before_drive_start(decoded_lines):
    decoded_lines = iter(decoded_lines)
    for line in decoded_lines:
        if line['topic'] == 'drive/start':
            break
    for line in decoded_lines:
        yield line

def decode_line(number, line):
    try:
        return json.loads(line)
    except json.JSONDecodeError:
        print(f'Error on line {number}:', repr(line))
        raise

def load_data_from_file(file_name):
    with open(file_name) as f:
        enumerated_lines = enumerate(line for line in f)
        populated_lines = ((number, line) for number, line in enumerated_lines if line != '\n')
        decoded_lines = [decode_line(number, line) for number, line in populated_lines]
    after_start = truncate_data_before_drive_start(decoded_lines)
    up_to_stop = list(truncate_data_after_drive_stop(after_start))
    # Normalise the times
    start_time = up_to_stop[0]['timestamp']
    for d in up_to_stop:
        d['timestamp'] -= start_time
    return up_to_stop


In [72]:
show(column(return_encoders_figure(data2), return_motors_figure(data2)))

In [73]:
data3 = load_data_from_file('data3.jsonl')
show(column(return_encoders_figure(data3), return_motors_figure(data3)))

In [75]:
data4 = load_data_from_file('data4.jsonl')
show(column(return_encoders_figure(data4), return_motors_figure(data4)))

In [76]:
data5 = load_data_from_file('data5.jsonl')
show(column(return_encoders_figure(data5), return_motors_figure(data5)))

In [79]:
data6 = load_data_from_file('data6.jsonl')
show(column(return_encoders_figure(data6), return_motors_figure(data6)))

In [81]:
data7 = load_data_from_file('data7.jsonl')
show(column(return_encoders_figure(data7), return_motors_figure(data7)))